In [2]:
'''
Written By: Abhijeet Kulkarni (amkulk@udel.edu)
For: Research Project MEEG678
Title: Autonomous driving using Model Predictive ContouringControl.

#Run this file if .dill files are not present.
'''

import sympy as sm
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import dill
dill.settings['recurse'] = True
from sympy.utilities.lambdify import lambdify, implemented_function
from HelpingFunctions import *
#!which python

'\nWritten By: Abhijeet Kulkarni (amkulk@udel.edu)\nFor: Research Project MEEG678\nTitle: Autonomous driving using Model Predictive ContouringControl.\n\n#Run this file if .dill files are not present.\n'

In [3]:
M,I,Fx1,Fy1,Fx2,Fy2,a1,a2,C1,C2 = sm.symbols(r'M I F_{x1} F_{y1} F_{x2} F_{y2} a_1 a_2 C_1 C_2',real=True)
vx,vy,dtheta,theta,delta = sm.symbols(r'v_x v_y \dot{\theta} \theta \delta',real=True)
X,Y,dX,dY = sm.symbols(r'X Y \dot{X} \dot{Y}')

In [4]:
#Velocities
vx = dX*sm.cos(theta) + dY*sm.sin(theta)
vy = -dX*sm.sin(theta) + dY*sm.cos(theta)
#Forces
#Fy1 = -C1*sm.atan((vy+dtheta*a1)/vx)
#Fy2 = -C2*sm.atan((vy-dtheta*a2)/vx)
Fy = Fy2 + Fy1*sm.cos(delta)+Fx1*sm.sin(delta)
Fx = Fx2 + Fx1*sm.cos(delta) - Fy1*sm.sin(delta)
Tau = -Fy2*a2+a1*(Fy1*sm.cos(delta)+Fx1*sm.sin(delta))

#Accelerations in Global frame
ddX = (Fx*sm.cos(theta)-Fy*sm.sin(theta))/M
ddY = (Fx*sm.sin(theta)+Fy*sm.cos(theta))/M
ddtheta = Tau/I
ddX=sm.simplify(ddX)
ddY=sm.simplify(ddY)
ddtheta=sm.simplify(ddtheta)
ode=sm.simplify(sm.Matrix([dX, dY, dtheta, ddX, ddY, ddtheta]))
fode=lambdify([[X,Y,theta,dX,dY,dtheta],[delta,Fx1,Fx2],[Fy1,Fy2],M,I,a1,a2],ode,'numpy')
ode
dill.dump(fode, open("eta_dot.dill","wb"))
print(sm.latex(ode))

Matrix([
[                                                                                                \dot{X}],
[                                                                                                \dot{Y}],
[                                                                                           \dot{\theta}],
[(F_{x1}*cos(\delta + \theta) + F_{x2}*cos(\theta) - F_{y1}*sin(\delta + \theta) - F_{y2}*sin(\theta))/M],
[(F_{x1}*sin(\delta + \theta) + F_{x2}*sin(\theta) + F_{y1}*cos(\delta + \theta) + F_{y2}*cos(\theta))/M],
[                                        (-F_{y2}*a_2 + a_1*(F_{x1}*sin(\delta) + F_{y1}*cos(\delta)))/I]])

\left[\begin{matrix}\dot{X}\\\dot{Y}\\\dot{\theta}\\\frac{F_{x1} \cos{\left(\delta + \theta \right)} + F_{x2} \cos{\left(\theta \right)} - F_{y1} \sin{\left(\delta + \theta \right)} - F_{y2} \sin{\left(\theta \right)}}{M}\\\frac{F_{x1} \sin{\left(\delta + \theta \right)} + F_{x2} \sin{\left(\theta \right)} + F_{y1} \cos{\left(\delta + \theta \right)} + F_{y2} \cos{\left(\theta \right)}}{M}\\\frac{- F_{y2} a_{2} + a_{1} \left(F_{x1} \sin{\left(\delta \right)} + F_{y1} \cos{\left(\delta \right)}\right)}{I}\end{matrix}\right]


In [5]:
#linearizing contact forces
q = sm.Matrix([X,Y,theta,dX,dY,dtheta]) #  states
u = sm.Matrix([delta, Fx1,Fx2]) #  inputs

D1,D2,C1,C2,B1,B2 = sm.symbols(r'D_1 D_2 C_1 C_2 B_1 B_2',real =True)
vx = dX*sm.cos(theta) + dY*sm.sin(theta)
vy = -dX*sm.sin(theta) + dY*sm.cos(theta)
alpha1 = -delta + sm.atan2((vy+dtheta*a1),vx)
alpha2 = sm.atan2((vy-dtheta*a2),vx)
sFy1 = -D1*sm.sin(C1*sm.atan(B1*alpha1))
sFy2 = -D2*sm.sin(C2*sm.atan(B2*alpha2))
sm.Matrix([sFy1,sFy2])
print(sm.latex(sm.Matrix([sFy1,sFy2])))



Matrix([
[-D_1*sin(C_1*atan(B_1*(-\delta + atan2(-\dot{X}*sin(\theta) + \dot{Y}*cos(\theta) + \dot{\theta}*a_1, \dot{X}*cos(\theta) + \dot{Y}*sin(\theta)))))],
[            -D_2*sin(C_2*atan(B_2*atan2(-\dot{X}*sin(\theta) + \dot{Y}*cos(\theta) - \dot{\theta}*a_2, \dot{X}*cos(\theta) + \dot{Y}*sin(\theta))))]])

\left[\begin{matrix}- D_{1} \sin{\left(C_{1} \operatorname{atan}{\left(B_{1} \left(- \delta + \operatorname{atan_{2}}{\left(- \dot{X} \sin{\left(\theta \right)} + \dot{Y} \cos{\left(\theta \right)} + \dot{\theta} a_{1},\dot{X} \cos{\left(\theta \right)} + \dot{Y} \sin{\left(\theta \right)} \right)}\right) \right)} \right)}\\- D_{2} \sin{\left(C_{2} \operatorname{atan}{\left(B_{2} \operatorname{atan_{2}}{\left(- \dot{X} \sin{\left(\theta \right)} + \dot{Y} \cos{\left(\theta \right)} - \dot{\theta} a_{2},\dot{X} \cos{\left(\theta \right)} + \dot{Y} \sin{\left(\theta \right)} \right)} \right)} \right)}\end{matrix}\right]


In [6]:
# Linear approximation of alpha2
alpha2Grad = sm.Matrix([alpha2]).jacobian(q)
alpha2Grad@q
alpha2
alpha2C = sm.Matrix([alpha2]) - alpha2Grad@q
falpha2data=lambdify([[X,Y,theta,dX,dY,dtheta],a1,a2],[alpha2C,alpha2Grad],'numpy')
dill.dump(falpha2data, open("alpha2data.dill","wb"))

Matrix([[\dot{X}*(-(\dot{X}*cos(\theta) + \dot{Y}*sin(\theta))*sin(\theta)/((\dot{X}*cos(\theta) + \dot{Y}*sin(\theta))**2 + (-\dot{X}*sin(\theta) + \dot{Y}*cos(\theta) - \dot{\theta}*a_2)**2) + (\dot{X}*sin(\theta) - \dot{Y}*cos(\theta) + \dot{\theta}*a_2)*cos(\theta)/((\dot{X}*cos(\theta) + \dot{Y}*sin(\theta))**2 + (-\dot{X}*sin(\theta) + \dot{Y}*cos(\theta) - \dot{\theta}*a_2)**2)) + \dot{Y}*((\dot{X}*cos(\theta) + \dot{Y}*sin(\theta))*cos(\theta)/((\dot{X}*cos(\theta) + \dot{Y}*sin(\theta))**2 + (-\dot{X}*sin(\theta) + \dot{Y}*cos(\theta) - \dot{\theta}*a_2)**2) + (\dot{X}*sin(\theta) - \dot{Y}*cos(\theta) + \dot{\theta}*a_2)*sin(\theta)/((\dot{X}*cos(\theta) + \dot{Y}*sin(\theta))**2 + (-\dot{X}*sin(\theta) + \dot{Y}*cos(\theta) - \dot{\theta}*a_2)**2)) - \dot{\theta}*a_2*(\dot{X}*cos(\theta) + \dot{Y}*sin(\theta))/((\dot{X}*cos(\theta) + \dot{Y}*sin(\theta))**2 + (-\dot{X}*sin(\theta) + \dot{Y}*cos(\theta) - \dot{\theta}*a_2)**2) + \theta*((-\dot{X}*sin(\theta) + \dot{Y}*cos(\th

atan2(-\dot{X}*sin(\theta) + \dot{Y}*cos(\theta) - \dot{\theta}*a_2, \dot{X}*cos(\theta) + \dot{Y}*sin(\theta))

In [7]:
# Linearizing 
f = (ode.subs([(Fy1,sFy1),(Fy2,sFy2)]))
f
A = (f.jacobian(q))
2
B = (f.jacobian(u))
# dq ~ f(q_,u_) + A(q-q_) + B(u-u_) =  Aq + Bu + (f(q_,u_) - Aq_ - Bu_) = Aq + Bu + G

G = f - A@q -B@u 


Matrix([
[                                                                                                                                                                                                                                                                                                                                                                             \dot{X}],
[                                                                                                                                                                                                                                                                                                                                                                             \dot{Y}],
[                                                                                                                                                                                                                                              

2

In [8]:
#Discretizing
# (qk+1 - qk)/dt = Akqk + Bkuk  + Gk
# qk+1 = qk + (Akqk + Bkuk + Gk)dt = (I + Ak)qk + (Bkdt)uk + Gkdt = (Ak)qk + (Bk)uk + Gk
dt = sm.symbols(r'd\text{t}',real = True) #time step
Ak = (sm.eye(A.shape[0]) + A*dt)
Bk = (B*dt)
Gk = (G*dt)
Ak
DiscDynaMatrices=lambdify([dt,[X,Y,theta,dX,dY,dtheta],[delta,Fx1,Fx2],M,I,a1,a2,D1,D2,C1,C2,B1,B2],[Ak,Bk,Gk],'numpy')
dill.dump(DiscDynaMatrices, open("DiscDynaMatrices.dill","wb"))

In [9]:
# error contouring and lag
ql,qc,ta = sm.symbols(r'q_l q_c t_a',real = True)
Xrefta,Yrefta,dXrefta,dYrefta,ddXrefta,ddYrefta = sm.symbols(r'X^{ref} Y^{ref} \dot{X}^{ref} \dot{Y}^{ref} \ddot{X}^{ref} \ddot{Y}^{ref}',real = True)
phita = sm.atan2(dYrefta,dXrefta)
#phitaa = sm.symbols(r'\Phi(t_a)',real=True)
ec = sm.Matrix([sm.sin(phita)*(X-Xrefta) - sm.cos(phita)*(Y-Yrefta)]) #contouring error
el = sm.Matrix([-sm.cos(phita)*(X-Xrefta) - sm.sin(phita)*(Y-Yrefta)]) #lag error
e = sm.Matrix([[ec],[el]])
print(sm.latex(e))
# linearizing f(x)~f(x0)+Jf(x0)*(x-x0)=(f(x0)-Jf(x0)*x0)+Jf(x0)*x)=C+Grad*x

var = sm.Matrix([Xrefta,Yrefta,dXrefta,dYrefta])
# Chain differentiation. To not fix the bezier polynomial in symbolic generation
Gradenom = sm.simplify(e.jacobian(q))
Gradenom
# Gradient while condidering [Xrefta,Yrefta,dXrefta,dYrefta]
Graderef  = e.jacobian(var)
#Graderef
#Common jacobian for reference var to parameter ta
JacobRP =sm.Matrix([[dXrefta],[dYrefta],[ddXrefta],[ddYrefta]]) 

#Gradient while considering [ta]
Gradeta  = sm.simplify(Graderef@JacobRP)
Gradeta

#Constant part of linearization
C  = sm.simplify(e  - Gradenom@q - Gradeta@sm.Matrix([ta]))
C


\left[\begin{matrix}- \frac{\dot{X}^{ref} \left(Y - Y^{ref}\right)}{\sqrt{\left(\dot{X}^{ref}\right)^{2} + \left(\dot{Y}^{ref}\right)^{2}}} + \frac{\dot{Y}^{ref} \left(X - X^{ref}\right)}{\sqrt{\left(\dot{X}^{ref}\right)^{2} + \left(\dot{Y}^{ref}\right)^{2}}}\\- \frac{\dot{X}^{ref} \left(X - X^{ref}\right)}{\sqrt{\left(\dot{X}^{ref}\right)^{2} + \left(\dot{Y}^{ref}\right)^{2}}} - \frac{\dot{Y}^{ref} \left(Y - Y^{ref}\right)}{\sqrt{\left(\dot{X}^{ref}\right)^{2} + \left(\dot{Y}^{ref}\right)^{2}}}\end{matrix}\right]


Matrix([
[ \dot{Y}^{ref}/sqrt(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2), -\dot{X}^{ref}/sqrt(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2), 0, 0, 0, 0],
[-\dot{X}^{ref}/sqrt(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2), -\dot{Y}^{ref}/sqrt(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2), 0, 0, 0, 0]])

Matrix([
[                                          (-\ddot{X}^{ref}*(\dot{X}^{ref}*(\dot{X}^{ref}*(-Y + Y^{ref}) + \dot{Y}^{ref}*(X - X^{ref})) + (Y - Y^{ref})*(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)) + \ddot{Y}^{ref}*(\dot{Y}^{ref}*(\dot{X}^{ref}*(Y - Y^{ref}) + \dot{Y}^{ref}*(-X + X^{ref})) + (X - X^{ref})*(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)))/(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)**(3/2)],
[(\ddot{X}^{ref}*(\dot{X}^{ref}*(\dot{X}^{ref}*(X - X^{ref}) + \dot{Y}^{ref}*(Y - Y^{ref})) + (-X + X^{ref})*(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)) + \ddot{Y}^{ref}*(\dot{Y}^{ref}*(\dot{X}^{ref}*(X - X^{ref}) + \dot{Y}^{ref}*(Y - Y^{ref})) + (-Y + Y^{ref})*(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)) + (\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)**2)/(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)**(3/2)]])

Matrix([
[                                          (t_a*(\ddot{X}^{ref}*(-\dot{X}^{ref}*(\dot{X}^{ref}*(Y - Y^{ref}) - \dot{Y}^{ref}*(X - X^{ref})) + (Y - Y^{ref})*(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)) - \ddot{Y}^{ref}*(\dot{Y}^{ref}*(\dot{X}^{ref}*(Y - Y^{ref}) - \dot{Y}^{ref}*(X - X^{ref})) + (X - X^{ref})*(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2))) + (\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)*(-X*\dot{Y}^{ref} + Y*\dot{X}^{ref} - \dot{X}^{ref}*(Y - Y^{ref}) + \dot{Y}^{ref}*(X - X^{ref})))/(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)**(3/2)],
[(-t_a*(\ddot{X}^{ref}*(\dot{X}^{ref}*(\dot{X}^{ref}*(X - X^{ref}) + \dot{Y}^{ref}*(Y - Y^{ref})) - (X - X^{ref})*(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)) + \ddot{Y}^{ref}*(\dot{Y}^{ref}*(\dot{X}^{ref}*(X - X^{ref}) + \dot{Y}^{ref}*(Y - Y^{ref})) - (Y - Y^{ref})*(\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)) + (\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)**2) + (\dot{X}^{ref}**2 + \dot{Y}^{ref}**2)*(X*\dot{X}^{ref} + Y*\dot{Y}^{ref} - \dot{X}^{ref}*(X - X^{ref}) - \dot{Y}^{ref

In [10]:
ErrorLinMat=lambdify([[X,Y,theta,dX,dY,dtheta],ta,Xrefta,Yrefta,dXrefta,dYrefta,ddXrefta,ddYrefta],(C,Gradenom,Gradeta),'numpy')
dill.dump(ErrorLinMat, open("ErrorsLindata.dill","wb"))
#C0,gradenom0,gradeta0=ErrorLinMat(np.array([1,1,1,1,1,1]),1,1,2,3,4,5,6)

array([[ 0.336],
       [-2.752]])

array([[ 0.8, -0.6,  0. ,  0. ,  0. ,  0. ],
       [-0.6, -0.8,  0. ,  0. ,  0. ,  0. ]])

array([[0.064],
       [4.952]])